# Task B3
Write a python program that will read the data from Fire data-Part1 and Climate data - Part1 and load them to the new database created in Task B2.

In [2]:
import csv
firePath = './FireData-Part1.csv'
climatePath = './ClimateData-Part1.csv'

In [3]:
def read_to_list(path):
    with open(path,'r') as f:
        reader = csv.reader(f)
        return list(reader)

In [4]:
climateData = read_to_list(climatePath)[1:]
fireData = read_to_list(firePath)[1:]

In [5]:
#make connection to mongod
import pymongo
from pymongo import MongoClient
client = MongoClient ()
#get the database
db = client.fit5148_A2
#create two new collections 
Fire = db.Fire
Climate = db.Climate

In [6]:
#create fires collection and insert fire documents into fires collection 
for fire in fireData:
    Fire.insert_one(
        {"Latitude": float(fire[0].strip()),
         "Longitude": float(fire[1].strip()),
         "Surface Temperature (kelvin)": float(fire[2].strip()),  
         "Datetime": fire[3].strip(),  
         "Power": float(fire[4].strip()),   
         "Confidence": int(fire[5].strip()),  
         "Date": fire[6].strip(),  
         "Surface Temperature (Celcius)":  int(fire[7].strip()),  
        })


In [7]:
#create climates collection and insert climate documents into climates collection
for climate in climateData:
    Climate.insert_one(
        {"Station": int(climate[0].strip()),
         "Date": climate[1].strip(),
         "Air Temperature(Celcius)": int(climate[2].strip()),  
         "Relative Humidity": float(climate[3].strip()),  
         "WindSpeed  (knots)": float(climate[4].strip()),   
         "Max Wind Speed": float(climate[5].strip()),  
         "MAX": climate[6].strip(),  
         "MIN": climate[7].strip(),  
         "Precipitation": climate[8].strip()
        })

In [8]:
#find the array of all associated object ids for referencing the child 
#and update the array for each climate document
#get all fire documents from fire collection
allFires = Fire.find({})
#get all climate documents from climate collection
allClimates = Climate.find({})
fireIds = []
for climate in allClimates:       
    results = Fire.find({"Date": climate['Date']})
    for doc in results:
        fireIds.append(doc['_id'])
    Climate.update_one({"_id": climate.get('_id')}, {"$set": {"fires": fireIds}}) 
    fireIds = []

# Task B4
Write queries to answer the Tasks A2-A8 on the new database. You need to write the queries as a python program using pymongo library in Jupyter Notebook. Please choose ONE of the Task A2-A8 and implement it in parallel.

### A2 Find climate data on 15th December 2017

In [9]:
from pprint import pprint
results = Climate.find({"Date":"12/15/2017"})
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 18,
 'Date': '12/15/2017',
 'MAX': '74.5*',
 'MIN': '53.1',
 'Max Wind Speed': 14.0,
 'Precipitation': '0.00I',
 'Relative Humidity': 52.0,
 'Station': 948702,
 'WindSpeed  (knots)': 7.1,
 '_id': ObjectId('5aee8c72adc24c1eb474a6a1'),
 'fires': [ObjectId('5aee8c70adc24c1eb4749aef'),
           ObjectId('5aee8c70adc24c1eb4749af0'),
           ObjectId('5aee8c70adc24c1eb4749af1'),
           ObjectId('5aee8c70adc24c1eb4749af2')]}


### A3 Find the latitude, longitude and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [10]:
results = Fire.find(
    {"Surface Temperature (Celcius)":{"$gt":65,"$lt":100}},
    {"Longitude":1,"Latitude":1,"Confidence":1,"_id":0}
)
for document in results: 
    pprint(document)

{'Confidence': 78, 'Latitude': -37.966, 'Longitude': 145.051}
{'Confidence': 86, 'Latitude': -35.543, 'Longitude': 143.316}
{'Confidence': 93, 'Latitude': -37.875, 'Longitude': 142.51}
{'Confidence': 95, 'Latitude': -37.613, 'Longitude': 149.305}
{'Confidence': 90, 'Latitude': -37.624, 'Longitude': 149.314}
{'Confidence': 93, 'Latitude': -38.057, 'Longitude': 144.211}
{'Confidence': 92, 'Latitude': -37.95, 'Longitude': 142.366}
{'Confidence': 100, 'Latitude': -36.282, 'Longitude': 146.157}
{'Confidence': 100, 'Latitude': -37.634, 'Longitude': 149.237}
{'Confidence': 98, 'Latitude': -37.605, 'Longitude': 149.302}
{'Confidence': 99, 'Latitude': -37.6, 'Longitude': 149.325}
{'Confidence': 95, 'Latitude': -37.618, 'Longitude': 149.281}
{'Confidence': 100, 'Latitude': -37.606, 'Longitude': 149.312}
{'Confidence': 97, 'Latitude': -37.61, 'Longitude': 149.307}
{'Confidence': 99, 'Latitude': -37.609, 'Longitude': 149.32}
{'Confidence': 93, 'Latitude': -37.945, 'Longitude': 144.354}
{'Confidenc

### A4 Find surface temperature (°C), air temperature (°C), relative humidity and maximum wind speed on 15th and 16th of December 2017

In [11]:
results = Fire.aggregate([
{"$lookup":{
    "from":"Climate",
    "localField":"Date",
    "foreignField":"Date",
    "as":"ClimateData"}
},
{
 "$replaceRoot": { "newRoot": { "$mergeObjects": [ { "$arrayElemAt": [ "$ClimateData", 0 ] }, "$$ROOT" ] } }
},

{
    "$project": {"Date":1,"Surface Temperature (Celcius)": 1,
               "Air Temperature(Celcius)":1,"Relative Humidity":1,
               "Max Wind Speed":1,"_id":0} 
},
{
    "$match":{"$or":[{"Date":"12/15/2017"},{"Date":"12/16/2017"}]}
}
])
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 18,
 'Date': '12/16/2017',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 43}
{'Air Temperature(Celcius)': 18,
 'Date': '12/16/2017',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 33}
{'Air Temperature(Celcius)': 18,
 'Date': '12/16/2017',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 54}
{'Air Temperature(Celcius)': 18,
 'Date': '12/16/2017',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 73}
{'Air Temperature(Celcius)': 18,
 'Date': '12/16/2017',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 18,
 'Date': '12/16/2017',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 75}
{'Air Temperature(Celcius)': 18,
 'Date': '12/16/2017',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface

### A5 Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [16]:
results = Fire.aggregate([
{"$lookup":{
    "from":"Climate",
    "localField":"Date",
    "foreignField":"Date",
    "as":"ClimateData"}
},
{
 "$replaceRoot": { "newRoot": { "$mergeObjects": [ { "$arrayElemAt": [ "$ClimateData", 0 ] }, "$$ROOT" ] } }
},
{ 
    "$project": {"Datetime":1,"Air Temperature(Celcius)":1,
	"Surface Temperature (Celcius)": 1,"Confidence":1, "_id":0}
 },
{
    "$match":{"Confidence":{"$gt":80,"$lt":100}}}

])
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 28,
 'Confidence': 82,
 'Datetime': '2017-12-27T00:02:15',
 'Surface Temperature (Celcius)': 63}
{'Air Temperature(Celcius)': 28,
 'Confidence': 86,
 'Datetime': '2017-12-27T00:02:14',
 'Surface Temperature (Celcius)': 67}
{'Air Temperature(Celcius)': 18,
 'Confidence': 94,
 'Datetime': '2017-12-16T15:38:39',
 'Surface Temperature (Celcius)': 43}
{'Air Temperature(Celcius)': 18,
 'Confidence': 93,
 'Datetime': '2017-12-16T04:35:13',
 'Surface Temperature (Celcius)': 73}
{'Air Temperature(Celcius)': 18,
 'Confidence': 84,
 'Datetime': '2017-12-16T04:34:58',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 18,
 'Confidence': 95,
 'Datetime': '2017-12-16T04:34:58',
 'Surface Temperature (Celcius)': 75}
{'Air Temperature(Celcius)': 18,
 'Confidence': 90,
 'Datetime': '2017-12-16T04:34:57',
 'Surface Temperature (Celcius)': 66}
{'Air Temperature(Celcius)': 18,
 'Confidence': 93,
 'Datetime': '2017-12-16T00:21:12',
 'Surface Temperature (Celcius

{'Air Temperature(Celcius)': 6,
 'Confidence': 84,
 'Datetime': '2017-07-02T04:29:18',
 'Surface Temperature (Celcius)': 52}
{'Air Temperature(Celcius)': 6,
 'Confidence': 88,
 'Datetime': '2017-07-02T04:29:00',
 'Surface Temperature (Celcius)': 58}
{'Air Temperature(Celcius)': 6,
 'Confidence': 88,
 'Datetime': '2017-07-02T04:29:00',
 'Surface Temperature (Celcius)': 58}
{'Air Temperature(Celcius)': 5,
 'Confidence': 81,
 'Datetime': '2017-06-30T00:27:19',
 'Surface Temperature (Celcius)': 48}
{'Air Temperature(Celcius)': 5,
 'Confidence': 81,
 'Datetime': '2017-06-30T00:27:19',
 'Surface Temperature (Celcius)': 48}
{'Air Temperature(Celcius)': 11,
 'Confidence': 91,
 'Datetime': '2017-06-20T04:02:00',
 'Surface Temperature (Celcius)': 68}
{'Air Temperature(Celcius)': 11,
 'Confidence': 97,
 'Datetime': '2017-06-20T04:02:00',
 'Surface Temperature (Celcius)': 80}
{'Air Temperature(Celcius)': 11,
 'Confidence': 94,
 'Datetime': '2017-06-14T04:43:00',
 'Surface Temperature (Celcius)': 8

 'Confidence': 81,
 'Datetime': '2017-04-23T05:02:50',
 'Surface Temperature (Celcius)': 54}
{'Air Temperature(Celcius)': 18,
 'Confidence': 87,
 'Datetime': '2017-04-22T04:20:20',
 'Surface Temperature (Celcius)': 62}
{'Air Temperature(Celcius)': 20,
 'Confidence': 84,
 'Datetime': '2017-04-20T04:44:20',
 'Surface Temperature (Celcius)': 58}
{'Air Temperature(Celcius)': 20,
 'Confidence': 93,
 'Datetime': '2017-04-20T04:34:10',
 'Surface Temperature (Celcius)': 72}
{'Air Temperature(Celcius)': 20,
 'Confidence': 88,
 'Datetime': '2017-04-20T04:33:20',
 'Surface Temperature (Celcius)': 63}
{'Air Temperature(Celcius)': 20,
 'Confidence': 90,
 'Datetime': '2017-04-20T04:32:50',
 'Surface Temperature (Celcius)': 67}
{'Air Temperature(Celcius)': 20,
 'Confidence': 85,
 'Datetime': '2017-04-20T04:32:30',
 'Surface Temperature (Celcius)': 60}
{'Air Temperature(Celcius)': 20,
 'Confidence': 98,
 'Datetime': '2017-04-20T04:32:30',
 'Surface Temperature (Celcius)': 84}
{'Air Temperature(Celcius

 'Surface Temperature (Celcius)': 56}
{'Air Temperature(Celcius)': 12,
 'Confidence': 82,
 'Datetime': '2017-04-15T04:15:10',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 12,
 'Confidence': 88,
 'Datetime': '2017-04-15T04:15:10',
 'Surface Temperature (Celcius)': 63}
{'Air Temperature(Celcius)': 12,
 'Confidence': 84,
 'Datetime': '2017-04-15T04:15:10',
 'Surface Temperature (Celcius)': 58}
{'Air Temperature(Celcius)': 12,
 'Confidence': 86,
 'Datetime': '2017-04-15T04:15:10',
 'Surface Temperature (Celcius)': 61}
{'Air Temperature(Celcius)': 12,
 'Confidence': 91,
 'Datetime': '2017-04-15T04:15:10',
 'Surface Temperature (Celcius)': 69}
{'Air Temperature(Celcius)': 12,
 'Confidence': 89,
 'Datetime': '2017-04-15T04:15:10',
 'Surface Temperature (Celcius)': 65}
{'Air Temperature(Celcius)': 12,
 'Confidence': 81,
 'Datetime': '2017-04-15T04:15:00',
 'Surface Temperature (Celcius)': 54}
{'Air Temperature(Celcius)': 12,
 'Confidence': 81,
 'Datetime': '2017-04-15T04:

 'Datetime': '2017-04-12T03:44:40',
 'Surface Temperature (Celcius)': 73}
{'Air Temperature(Celcius)': 14,
 'Confidence': 81,
 'Datetime': '2017-04-11T04:39:20',
 'Surface Temperature (Celcius)': 54}
{'Air Temperature(Celcius)': 14,
 'Confidence': 84,
 'Datetime': '2017-04-11T04:39:00',
 'Surface Temperature (Celcius)': 58}
{'Air Temperature(Celcius)': 14,
 'Confidence': 85,
 'Datetime': '2017-04-11T04:39:00',
 'Surface Temperature (Celcius)': 60}
{'Air Temperature(Celcius)': 19,
 'Confidence': 99,
 'Datetime': '2017-04-08T04:14:00',
 'Surface Temperature (Celcius)': 85}
{'Air Temperature(Celcius)': 19,
 'Confidence': 91,
 'Datetime': '2017-04-08T04:12:50',
 'Surface Temperature (Celcius)': 69}
{'Air Temperature(Celcius)': 19,
 'Confidence': 98,
 'Datetime': '2017-04-08T04:08:50',
 'Surface Temperature (Celcius)': 83}
{'Air Temperature(Celcius)': 19,
 'Confidence': 90,
 'Datetime': '2017-04-08T04:08:30',
 'Surface Temperature (Celcius)': 66}
{'Air Temperature(Celcius)': 21,
 'Confidenc

{'Air Temperature(Celcius)': 18,
 'Confidence': 88,
 'Datetime': '2017-03-28T04:28:30',
 'Surface Temperature (Celcius)': 63}
{'Air Temperature(Celcius)': 18,
 'Confidence': 87,
 'Datetime': '2017-03-28T04:28:30',
 'Surface Temperature (Celcius)': 62}
{'Air Temperature(Celcius)': 18,
 'Confidence': 97,
 'Datetime': '2017-03-28T04:27:40',
 'Surface Temperature (Celcius)': 80}
{'Air Temperature(Celcius)': 18,
 'Confidence': 82,
 'Datetime': '2017-03-28T04:27:00',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 18,
 'Confidence': 99,
 'Datetime': '2017-03-28T04:27:00',
 'Surface Temperature (Celcius)': 86}
{'Air Temperature(Celcius)': 18,
 'Confidence': 81,
 'Datetime': '2017-03-28T04:26:50',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 18,
 'Confidence': 83,
 'Datetime': '2017-03-28T04:26:50',
 'Surface Temperature (Celcius)': 57}
{'Air Temperature(Celcius)': 18,
 'Confidence': 87,
 'Datetime': '2017-03-28T04:26:50',
 'Surface Temperature (Celcius

### A6 Find top 10 records with highest surface temperature (°C).

In [13]:
results = Fire.aggregate([
{"$lookup":{
    "from":"Climate",
    "localField":"Date",
    "foreignField":"Date",
    "as":"ClimateData"}
},
{
 "$replaceRoot": { "newRoot": { "$mergeObjects": [ { "$arrayElemAt": [ "$ClimateData", 0 ] }, "$$ROOT" ] } }
},
{ 
    "$project": { "ClimateData": 0,"_id":0}
},
{
    "$sort":{"Surface Temperature (Celcius)":-1}
},
{
    "$limit":10
}
])
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 15,
 'Confidence': 100,
 'Date': '4/18/2017',
 'Datetime': '2017-04-18T04:52:00',
 'Latitude': -38.1665,
 'Longitude': 143.062,
 'MAX': '69.8*',
 'MIN': '46.8',
 'Max Wind Speed': 9.9,
 'Power': 239.8,
 'Precipitation': '0.00I',
 'Relative Humidity': 56.1,
 'Station': 948701,
 'Surface Temperature (Celcius)': 124,
 'Surface Temperature (kelvin)': 397.5,
 'WindSpeed  (knots)': 5.1,
 'fires': [ObjectId('5aee8c71adc24c1eb4749f6d'),
           ObjectId('5aee8c71adc24c1eb4749f6e'),
           ObjectId('5aee8c71adc24c1eb4749f6f'),
           ObjectId('5aee8c71adc24c1eb4749f70'),
           ObjectId('5aee8c71adc24c1eb4749f71'),
           ObjectId('5aee8c71adc24c1eb4749f72'),
           ObjectId('5aee8c71adc24c1eb4749f73'),
           ObjectId('5aee8c71adc24c1eb4749f74'),
           ObjectId('5aee8c71adc24c1eb4749f75'),
           ObjectId('5aee8c71adc24c1eb4749f76'),
           ObjectId('5aee8c71adc24c1eb4749f77'),
           ObjectId('5aee8c71adc24c1eb4749f78'),

 'Station': 948701,
 'Surface Temperature (Celcius)': 120,
 'Surface Temperature (kelvin)': 393.7,
 'WindSpeed  (knots)': 5.9,
 'fires': [ObjectId('5aee8c71adc24c1eb474a141'),
           ObjectId('5aee8c71adc24c1eb474a142'),
           ObjectId('5aee8c71adc24c1eb474a143'),
           ObjectId('5aee8c71adc24c1eb474a144'),
           ObjectId('5aee8c71adc24c1eb474a145'),
           ObjectId('5aee8c71adc24c1eb474a146'),
           ObjectId('5aee8c71adc24c1eb474a147'),
           ObjectId('5aee8c71adc24c1eb474a148'),
           ObjectId('5aee8c71adc24c1eb474a149'),
           ObjectId('5aee8c71adc24c1eb474a14a'),
           ObjectId('5aee8c71adc24c1eb474a14b'),
           ObjectId('5aee8c71adc24c1eb474a14c'),
           ObjectId('5aee8c71adc24c1eb474a14d'),
           ObjectId('5aee8c71adc24c1eb474a14e'),
           ObjectId('5aee8c71adc24c1eb474a14f'),
           ObjectId('5aee8c71adc24c1eb474a150'),
           ObjectId('5aee8c71adc24c1eb474a151'),
           ObjectId('5aee8c71adc24c1eb47

### A7 Find the number of fire in each day. You are required to only display total number of fire and the date in the output

In [14]:
results = Fire.aggregate([
{"$group":{"_id":"$Date","NumberOfFire":{"$sum":1}}},
{"$project":{"Date":"$_id","NumberOfFire":1,"_id":0}}
])
for document in results: 
    pprint(document)

{'Date': '3/6/2017', 'NumberOfFire': 2}
{'Date': '3/7/2017', 'NumberOfFire': 1}
{'Date': '3/8/2017', 'NumberOfFire': 2}
{'Date': '3/10/2017', 'NumberOfFire': 8}
{'Date': '3/12/2017', 'NumberOfFire': 5}
{'Date': '3/13/2017', 'NumberOfFire': 2}
{'Date': '3/15/2017', 'NumberOfFire': 7}
{'Date': '3/18/2017', 'NumberOfFire': 3}
{'Date': '3/24/2017', 'NumberOfFire': 2}
{'Date': '3/25/2017', 'NumberOfFire': 13}
{'Date': '3/26/2017', 'NumberOfFire': 17}
{'Date': '3/28/2017', 'NumberOfFire': 54}
{'Date': '3/29/2017', 'NumberOfFire': 1}
{'Date': '4/1/2017', 'NumberOfFire': 7}
{'Date': '4/2/2017', 'NumberOfFire': 5}
{'Date': '4/4/2017', 'NumberOfFire': 89}
{'Date': '4/5/2017', 'NumberOfFire': 49}
{'Date': '4/7/2017', 'NumberOfFire': 39}
{'Date': '4/6/2017', 'NumberOfFire': 118}
{'Date': '4/8/2017', 'NumberOfFire': 20}
{'Date': '4/11/2017', 'NumberOfFire': 24}
{'Date': '4/12/2017', 'NumberOfFire': 69}
{'Date': '4/13/2017', 'NumberOfFire': 357}
{'Date': '4/17/2017', 'NumberOfFire': 38}
{'Date': '4/

### A8 Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output

In [15]:
results = Fire.aggregate([
{"$group":{"_id":"$Date", "AverageSurfaceTemp":{"$avg":"$Surface Temperature (Celcius)"}}},
{"$project":{"Date":"$_id","AverageSurfaceTemp":1,"_id":0}}
])
for document in results: 
    pprint(document)

{'AverageSurfaceTemp': 60.5, 'Date': '3/6/2017'}
{'AverageSurfaceTemp': 64.0, 'Date': '3/7/2017'}
{'AverageSurfaceTemp': 51.5, 'Date': '3/8/2017'}
{'AverageSurfaceTemp': 69.375, 'Date': '3/10/2017'}
{'AverageSurfaceTemp': 88.2, 'Date': '3/12/2017'}
{'AverageSurfaceTemp': 38.5, 'Date': '3/13/2017'}
{'AverageSurfaceTemp': 46.0, 'Date': '3/15/2017'}
{'AverageSurfaceTemp': 79.33333333333333, 'Date': '3/18/2017'}
{'AverageSurfaceTemp': 49.0, 'Date': '3/24/2017'}
{'AverageSurfaceTemp': 66.0, 'Date': '3/25/2017'}
{'AverageSurfaceTemp': 56.88235294117647, 'Date': '3/26/2017'}
{'AverageSurfaceTemp': 60.925925925925924, 'Date': '3/28/2017'}
{'AverageSurfaceTemp': 51.0, 'Date': '3/29/2017'}
{'AverageSurfaceTemp': 46.714285714285715, 'Date': '4/1/2017'}
{'AverageSurfaceTemp': 45.2, 'Date': '4/2/2017'}
{'AverageSurfaceTemp': 62.57303370786517, 'Date': '4/4/2017'}
{'AverageSurfaceTemp': 53.142857142857146, 'Date': '4/5/2017'}
{'AverageSurfaceTemp': 50.69230769230769, 'Date': '4/7/2017'}
{'AverageSur